In [1]:
from sqlalchemy import create_engine, Table, MetaData, insert, text, inspect
from sqlalchemy.orm import sessionmaker
import pandas as pd
import json

In [2]:
DATABASE_URL = f"mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)
metadata = MetaData()
paximum_table = Table('juniper', metadata, autoload_with=engine)

In [3]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT DATABASE()"))
        db_name = result.fetchone()[0]
        print(f"Connected to database: {db_name}")
except Exception as e:
    print(f"Error: {e}")


Connected to database: csvdata01_02102024


In [4]:
try:
    inspect_table = inspect(engine)
    tables = inspect_table.get_table_names()
    if tables:
        for table in tables:
            print(table)

except Exception as e:
    print(f"Error {e}")

hotels_info
hotels_info_with_gidestination_code
info_01
innova_hotels_main
juniper
my_table
paximum
ratehawk
z_ocas_member_data


In [13]:
import requests
import xmltodict
import json

url = "https://xml-uat.bookingengine.es/WebService/jp/operations/staticdatatransactions.asmx"

payload = "<soapenv:Envelope xmlns:soapenv=\"http://schemas.xmlsoap.org/soap/envelope/\" xmlns=\"http://www.juniper.es/webservice/2007/\">\n   <soapenv:Header/>\n   <soapenv:Body>\n      <HotelPortfolio>\n         <HotelPortfolioRQ Version=\"1.1\" Language=\"en\" Page=\"2\" RecordsPerPage=\"5\">\n            <Login Password=\"^wvW6#[49J/G\" Email=\"TestXMLNofshonTours\"/>\n         </HotelPortfolioRQ>\n      </HotelPortfolio>\n   </soapenv:Body>\n</soapenv:Envelope>"
headers = {
  'Content-Type': 'text/xml;charset=UTF-8',
  'SOAPAction': '"http://www.juniper.es/webservice/2007/HotelPortfolio"',
  'Cookie': 'StatC=8l+HAPXk1NeIsJR8qq/ISw==; StatP=lh5UxVPejVDjU3b27uc0fygTysSaay6K5gfbVxJWTYZM4f+TafxAFFOXsj/1uQ3n; idioma=en'
}

response = requests.request("POST", url, headers=headers, data=payload)

data_dict = xmltodict.parse(response.text)
print(data_dict)


{'soap:Envelope': {'@xmlns:soap': 'http://schemas.xmlsoap.org/soap/envelope/', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema', 'soap:Body': {'HotelPortfolioResponse': {'@xmlns': 'http://www.juniper.es/webservice/2007/', 'HotelPortfolioRS': {'@Url': 'http://xml-uat.bookingengine.es', '@TimeStamp': '2024-11-23T11:34:20.3714381+01:00', '@IntCode': 'FB5poqs4qMHFVbzEws9wODDtnSZiaOfoMhX3sI/X0WY=', 'HotelPortfolio': {'@Page': '2', '@TotalPages': '1952', '@TotalRecords': '9757', '@LastUpdated': '2024-09-05T20:00:59.153', 'Hotel': [{'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060468', '@HasSynonyms': 'false', 'Name': 'Embat', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer Nanses, 7, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5343124', 'Longitude': '2.7144459', 'HotelCategory': {'@Type': '2Est', '@Code': '2', '#text'

In [14]:
# Extract hotel data
hotels = []
hotel_list = data_dict.get("soap:Envelope", {}).get("soap:Body", {}).get(
    "HotelPortfolioResponse", {}
).get("HotelPortfolioRS", {}).get("HotelPortfolio", {}).get("Hotel", [])

# print(hotel_list)

if not hotel_list:
    print("No hotels found in the response.")
    exit()

# Process and insert hotel data into the database
with engine.connect() as conn:
    for hotel in hotel_list:
        hotel_data = {
            "HotelName": hotel.get("Name"),
            "HotelId": hotel.get("@JPCode"),
            "ZipCode": hotel.get("ZipCode"),
            "Latitude": hotel.get("Latitude"),
            "Longitude": hotel.get("Longitude"),
        }

        # Insert into the database
        try:
            stmt = insert(juniper_table).values(hotel_data)
            conn.execute(stmt)
            conn.commit()  # Save the changes immediately
            print(f"Inserted: {hotel_data}")
        except Exception as e:
            print(f"Error inserting {hotel_data}: {e}")

[{'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060468', '@HasSynonyms': 'false', 'Name': 'Embat', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer Nanses, 7, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5343124', 'Longitude': '2.7144459', 'HotelCategory': {'@Type': '2Est', '@Code': '2', '#text': '2 Stars'}, 'City': {'@Id': '15011', '@JPDCode': 'JPD054557', '#text': 'Palma de Mallorca'}}, {'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060489', '@HasSynonyms': 'false', 'Name': 'Isla del Sol Apartments', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer dels Pins, 5, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5375776', 'Longitude': '2.7142996', 'HotelCategory': {'@Code': '8'}, 'City': {'@Id': '15011', '@JPDCode': 'JPD054557', '#text': 'Palma de Mallorca'}}, {'@